<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/openAI_RESPONSIBLE_API_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai -q
!pip install colab-env -q

import colab_env
import asyncio
from openai import OpenAI
import os
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY")) #Using enviromental variable.

## OPENAI - without AGENT CLASS

In [24]:
class AircraftMaintenanceAgent:
    def __init__(self, aircraft_type="Boeing 787", engine_type="General Electric GEnx", api_key=None):
        self.context = {}
        self.aircraft_type = aircraft_type
        self.engine_type = engine_type
        if api_key:
            from openai import OpenAI
            self.client = OpenAI(api_key=api_key)
        else:
            import os
            from openai import OpenAI
            self.client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    async def observe(self, input_data):
        print("AMA Observing:", input_data)
        return input_data

    async def orient(self, observed_data):
        print("AMA Orienting:", observed_data)
        try:
            # More specific issue classification
            response = self.client.chat.completions.create(
                #model="gpt-4o",
                model="gpt-4.5-preview-2025-02-27",
                messages=[
                    {"role": "system", "content": f"Classify the following aircraft issue for a {self.aircraft_type} with a {self.engine_type} engine."},
                    {"role": "user", "content": observed_data},
                ],
            )
            issue_type = response.choices[0].message.content.strip()
            self.context['issue_type'] = issue_type

            # More specific maintenance info retrieval
            response2 = self.client.chat.completions.create(
                #model="gpt-4o",
                model="gpt-4.5-preview-2025-02-27",
                messages=[
                    {"role": "system", "content": f"Find relevant maintenance procedures for '{issue_type}' on a {self.aircraft_type} with a {self.engine_type} engine."},
                    {"role": "user", "content": "Search for maintenance procedures."},
                ],
            )
            manual_references = response2.choices[0].message.content.strip()
            self.context['manual_references'] = manual_references

            return observed_data

        except Exception as e:
            print(f"Error in orienting: {e}")
            return observed_data #Return the input, so the program doesn't crash.

    async def decide(self, oriented_data):
        print("AMA Deciding:", oriented_data)
        if self.context.get('issue_type'):
            return "Provide specific diagnostic instructions"
        else:
            return "Request more specific information"

    async def act(self, action, oriented_data):
        print("AMA Acting:", action)
        try:
            if action == "Provide specific diagnostic instructions":
                # Diagnostic instruction generation
                print("Generating diagnostic instructions...")
                print('\n')
                print(f"Aircraft Type: {self.aircraft_type}")
                print(f"Engine Type: {self.engine_type}")
                print(f"Issue Type: {self.context.get('issue_type')}")
                print(f"Manual References: {self.context.get('manual_references')}")
                print(f"Observed Data: {oriented_data}")
                print(f"Context: {self.context}")
                print(f"Action: {action}")
                print(f"Oriented Data: {oriented_data}")
                print('\n')
                response = self.client.chat.completions.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "system", "content": f"Generate step-by-step diagnostic and repair instructions for a {self.aircraft_type} with a {self.engine_type} engine experiencing: '{self.context.get('issue_type')}'"},
                        {"role": "user", "content": "Provide detailed instructions."},
                    ],
                )
                repair_steps = response.choices[0].message.content.strip()
                return repair_steps
            elif action == "Request more specific information":
                return f"Please provide more specific details about the issue on your {self.aircraft_type}'s {self.engine_type} engine."
            else:
                return "Action not recognized"
        except Exception as e:
            print(f"Error in acting: {e}")
            return "An error occurred during processing."

    async def process_input(self, input_data):
        observed = await self.observe(input_data)
        oriented = await self.orient(observed)
        action = await self.decide(oriented)
        result = await self.act(action, oriented)
        return result

async def main():
    ama = AircraftMaintenanceAgent() #Boeing 787 and General Electric GEnx are now defaults.
    pilot_report = "Engine making strange vibration during cruise, and oil temperature is slightly high."
    result = await ama.process_input(pilot_report)
    print('\n')
    print("AMA Response:", result)


# Instead of using asyncio.run, directly await the main function.
import asyncio
await main()

AMA Observing: Engine making strange vibration during cruise, and oil temperature is slightly high.
AMA Orienting: Engine making strange vibration during cruise, and oil temperature is slightly high.
AMA Deciding: Engine making strange vibration during cruise, and oil temperature is slightly high.
AMA Acting: Provide specific diagnostic instructions
Generating diagnostic instructions...


Aircraft Type: Boeing 787
Engine Type: General Electric GEnx
Issue Type: This issue can be classified as follows:

Aircraft Type: Boeing 787  
Engine Model: General Electric GEnx  
Phase of Flight: Cruise  
Issue Category: Powerplant / Engine System  
Symptoms:  
- Unusual vibrations from engine  
- Elevated oil temperature (slightly high)

Recommendation:  
Conduct a thorough inspection of the engine for potential rotor imbalance or component wear. Monitor engine oil system for contamination, leaks, or reduced effectiveness of cooling. Further troubleshooting procedures and engine health monitoring s

## OPENAI - AGENT CLASS

In [ ]:
!pip install openai-agents -q

In [32]:
from IPython import get_ipython
from IPython.display import display
# %%
!pip install openai -q
!pip install colab-env -q
!pip install openai-agents -q

import colab_env
import asyncio
from openai import OpenAI
from agents import Agent, Runner  # Importing Agent and Runner
import os
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY")) #Using enviromental variable.
# %%
class AircraftMaintenanceAgent:
    def __init__(self, aircraft_type="Boeing 787", engine_type="General Electric GEnx", api_key=None):
        self.context = {}
        self.aircraft_type = aircraft_type
        self.engine_type = engine_type
        if api_key:
            from openai import OpenAI
            self.client = OpenAI(api_key=api_key)
        else:
            import os
            from openai import OpenAI
            self.client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

    async def observe(self, input_data):
        print("AMA Observing:", input_data)
        return input_data

    async def orient(self, observed_data):
        print("AMA Orienting:", observed_data)
        try:
            # More specific issue classification
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",  # Changed model to gpt-3.5-turbo
                messages=[
                    {"role": "system", "content": f"Classify the following aircraft issue for a {self.aircraft_type} with a {self.engine_type} engine."},
                    {"role": "user", "content": observed_data},
                ],
            )
            issue_type = response.choices[0].message.content.strip()
            self.context['issue_type'] = issue_type

            # More specific maintenance info retrieval
            response2 = self.client.chat.completions.create(
                model="gpt-3.5-turbo",  # Changed model to gpt-3.5-turbo
                messages=[
                    {"role": "system", "content": f"Find relevant maintenance procedures for '{issue_type}' on a {self.aircraft_type} with a {self.engine_type} engine."},
                    {"role": "user", "content": "Search for maintenance procedures."},
                ],
            )
            manual_references = response2.choices[0].message.content.strip()
            self.context['manual_references'] = manual_references

            return observed_data

        except Exception as e:
            print(f"Error in orienting: {e}")
            return observed_data #Return the input, so the program doesn't crash.

    async def decide(self, oriented_data):
        print("AMA Deciding:", oriented_data)
        if self.context.get('issue_type'):
            return "Provide specific diagnostic instructions"
        else:
            return "Request more specific information"

    async def act(self, action, oriented_data):
        print("AMA Acting:", action)
        try:
            if action == "Provide specific diagnostic instructions":
                print("Generating diagnostic instructions...")
                print('\n')

                print(f"Aircraft Type: {self.aircraft_type}")
                print(f"Engine Type: {self.engine_type}")
                print(f"Issue Type: {self.context.get('issue_type')}")
                print(f"Manual References: {self.context.get('manual_references')}")
                print(f"Observed Data: {oriented_data}")
                print(f"Context: {self.context}")
                print(f"Oriented Data: {oriented_data}")
                print('\n')


                # Using Agent and Runner, running asynchronously:
                agent = Agent(
                    name="MaintenanceAssistant",
                    instructions=f"Generate step-by-step diagnostic and repair instructions for a {self.aircraft_type} with a {self.engine_type} engine experiencing: '{self.context.get('issue_type')}'",
                )
                # Run Runner.run asynchronously
                result = await Runner.run(agent, "Provide detailed instructions.")
                repair_steps = result.final_output
                return repair_steps

            elif action == "Request more specific information":
                return f"Please provide more specific details about the issue on your {self.aircraft_type}'s {self.engine_type} engine."
            else:
                return "Action not recognized"
        except Exception as e:
            print(f"Error in acting: {e}")
            return "An error occurred during processing."

    async def process_input(self, input_data):
        observed = await self.observe(input_data)
        oriented = await self.orient(observed)
        action = await self.decide(oriented)
        result = await self.act(action, oriented)
        return result

async def main():
    ama = AircraftMaintenanceAgent() #Boeing 787 and General Electric GEnx are now defaults.
    pilot_report = "Engine making strange vibration during cruise, and oil temperature is slightly high."
    result = await ama.process_input(pilot_report)
    print('\n')
    print("AMA Response:", result)


# Instead of using asyncio.run, directly await the main function.
import asyncio
await main()

AMA Observing: Engine making strange vibration during cruise, and oil temperature is slightly high.
AMA Orienting: Engine making strange vibration during cruise, and oil temperature is slightly high.
AMA Deciding: Engine making strange vibration during cruise, and oil temperature is slightly high.
AMA Acting: Provide specific diagnostic instructions
Generating diagnostic instructions...


Aircraft Type: Boeing 787
Engine Type: General Electric GEnx
Issue Type: Engine Mechanical/System Failure/Malfunction
Manual References: I'm unable to browse the internet in real-time to find specific maintenance procedures for a Boeing 787 with a General Electric GEnx engine. However, I can suggest some general steps that might be involved in addressing an engine mechanical/system failure or malfunction on an aircraft like the Boeing 787 with a GEnx engine. Here are some general steps that might be taken:

1. **Identify the Issue**: The first step is to identify and diagnose the specific issue with t